In [36]:
import numpy as np
import matplotlib.pyplot as plt
from classy import Class
from cobaya.run import run
import os, sys

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['mathtext.fontset'], plt.rcParams['font.family'] = 'stix', 'STIXGeneral'
plt.rcParams.update({'font.size': 15})

In [37]:
home = os.getcwd()
print('Current working directory:', home)

Current working directory: /mnt/c/Users/darby/OneDrive/Desktop/cosmo_dep_feedback


In [38]:
## Load in IllustrisTNG simulation parameters
lhc_TNG = np.loadtxt(home+'/latin_hypercube_params_IllustrisTNG.txt')

In [39]:
## Load in data 
# redshift, z = 0.00
k = np.load(home+'/Pk/IllustrisTNG_k_m_z=0.00.npy')
Pk = np.load(home+'/Pk/IllustrisTNG_Pk_m_z=0.00.npy')
Pk0 = Pk[0]

In [44]:
# Paco said k values less than 30 should be good so I will cut out those values.
## Jia goes up to k_max=10, so I will use that here.
index = np.where(k>10)[0]
ks0 = k[0:index[0]]
Pks0 = Pk0[0:index[0]]

ks0.shape, Pks0.shape

((39,), (39,))

In [41]:
Pks0_err = 0.10*Pks0

In [113]:
cosmo_params = {
    # Fixed
    'h': 0.6711, 
    'n_s': 0.9624, 
    'Omega_b': 0.049,
    'Omega_k': 0,
    'P_k_max_h/Mpc': 10,
    'lensing': 'no',
    'z_pk': 0, 
    'output': 'mPk',
    'non_linear':'hmcode'}


def classy_model(eta, c):
    model = Class()
    model.set({'eta_0': eta, 
               'c_min':c, 
               'h': 0.6711, 
               'n_s': 0.9624, 
               'Omega_b': 0.049,
               'Omega_k': 0,
               'P_k_max_h/Mpc': 10,
               'lensing': 'no',
               'z_pk': 0, 
               'output': 'mPk',
               'non_linear':'hmcode'})
    model.compute()
    
    k_model = np.linspace(min(ks0), 10, len(ks0))
    Pk_model = [] 
    h = model.h()
    for k in k_model:
        Pk = model.pk(k*h, 0)*h**3
        Pk_model.append(Pk) 
    Pk_model = np.array(Pk_model)
    sigma2 = Pks0_err**2
    return -0.5 * np.sum((Pks0 - Pk_model)**2 / sigma2 + np.log(sigma2))


info = {
    'likelihood': {'external': classy_model},
    'params': dict([
        ('eta', {
            'prior': {'min': -5, 'max': 5},
            'ref': 0.76,
            'latex': 'eta'}),
        ('c', {
            'prior': {'min': -5, 'max': 35},
            'ref': 2.32,
            'latex': 'c'})]),
#     'theory': {'classy': {'extra_args': cosmo_params}},
    'sampler': {
        'mcmc': {'max_tries':100}}
}


In [114]:
updated_info, sampler = run(info)

[external] Initialized external likelihood.
[mcmc] Getting initial point... (this may take a few seconds)
[model] Measuring speeds... (this may take a few seconds)
[model] Setting measured speeds (per sec): {external: 1.23}
[prior] *WARNING* Reference pdf not defined or improper for some parameters. Using prior's sigma instead for them.
[mcmc] Covariance matrix not present. We will start learning the covariance of the proposal earlier: R-1 = 30 (would be 2 if all params loaded).
[mcmc] Initial point: eta:0.76, c:2.32
[mcmc] Sampling!
[mcmc] Progress @ 2022-08-18 22:01:45 : 1 steps taken, and 0 accepted.
[mcmc] Progress @ 2022-08-18 22:02:45 : 110 steps taken, and 4 accepted.
[mcmc] Progress @ 2022-08-18 22:03:46 : 222 steps taken, and 7 accepted.
[mcmc] Progress @ 2022-08-18 22:04:46 : 337 steps taken, and 9 accepted.
[mcmc] *ERROR* The chain has been stuck for 100 attempts, stopping sampling. Make sure the reference point is semsible and initial covmat.For parameters not included in a

LoggedError: The chain has been stuck for 100 attempts, stopping sampling. Make sure the reference point is semsible and initial covmat.For parameters not included in an initial covmat, the 'proposal' width set for each parameter should be of order of the expected conditional posterior width, which may be much smaller than the marginalized posterior width - choose a smaller rather than larger value if in doubt. You can also decrease the 'proposal_scale' option for mcmc, though small values will sample less efficiently once things converge.
Alternatively (though not advisable) make 'max_tries: np.inf' (or 'max_tries: .inf' in yaml).
Current point: eta:1.768967, c:16.34555

In [77]:
updated_info, sampler = run?

In [21]:
# info = {
#     "likelihood": {
#         "gaussian_mixture": {
#             "means": [0.2, 0],
#             "covs": [[0.1, 0.05],
#                      [0.05, 0.2]],
#             "derived": True}},
#     "params": dict([
#         ("a", {
#             "prior": {"min": -0.5, "max": 3},
#             "latex": r"\alpha"}),
#         ("b", {
#             "prior": {"dist": "norm", "loc": 0, "scale": 1},
#             "ref": 0,
#             "proposal": 0.5,
#             "latex": r"\beta"}),
#         ("derived_a", {
#             "latex": r"\alpha^\prime"}),
#         ("derived_b", {
#             "latex": r"\beta^\prime"})]),
#     "sampler": {
#         "mcmc": None}}